In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import re

In [2]:
df = pd.read_csv('Capstone_data.csv')

In [3]:
len(df)

136581

In [4]:
df.head(5)
#df = df.drop(['Unnamed: 0'],axis=1)

,url,label
0,http://www.crestonwood.com/router.php,0
1,http://rgipt.ac.in,0
2,http://vamoaestudiarmedicina.blogspot.com/,0
3,https://www.astrologyonline.eu/Astro_MemoNew/P...,0
4,https://www.missfiga.com/,0


In [5]:
df['label'].value_counts()

1    68644
0    67937
Name: label, dtype: int64

In [6]:
df.isnull().sum() #no. of null values in each column

url      0
label    0
dtype: int64

In [7]:
df = df.dropna() #dropping all null values bcz.. null values in URL column can't be handled by filling any values

In [8]:
df.isnull().sum() #checking for null values after dropping them

url      0
label    0
dtype: int64

In [9]:
duplicate = df[df.duplicated()] 
  
print("Duplicate Rows :",duplicate)

Duplicate Rows :                                                       url  label
62533               http://www.crestonwood.com/router.php      0
62534                                  http://rgipt.ac.in      0
62535          http://vamoaestudiarmedicina.blogspot.com/      0
62536   https://www.astrologyonline.eu/Astro_MemoNew/P...      0
62537                           https://www.missfiga.com/      0
...                                                   ...    ...
136461                                      www.yahoo.com      0
136521                                 https://github.com      0
136570                                   www.wallmart.com      1
136571                                    www.amazons.com      1
136574                                    www.amazons.com      1

[5007 rows x 2 columns]


In [10]:
df.drop_duplicates(keep = 'first', inplace = True)

In [11]:
duplicate = df[df.duplicated()] 
  
print("Duplicate Rows :",duplicate)

Duplicate Rows : Empty DataFrame
Columns: [url, label]
Index: []


In [12]:
import string

char = set(string.printable) #set of letters which has ascii values

d = []
#Checking for those rows which have those characters which does not have ascii values or non-english characters in the url
for i in df.itertuples():
    url=str(i.url)
    for k in url: 
        if k not in char:
            print(i)
            d.append(i)
            break
        

In [13]:
for i in range(len(d)):
    df = df.drop(d[i][0])

In [14]:
#remove http
def remove_proto(url):
    pat = re.compile('(ftps://|https://|http://|ftp://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)')
    try:
        mat = re.match(pat,url)
        if (mat.group(1)):
            url = url.replace(mat.group(1), '')
            #print(url)
        else:
            pass
    except:
        print(url)
    return url
df['url'] = df['url'].apply(lambda row: remove_proto(row))

In [15]:
import re

#counting no. of numericals in the url and adding a column to the dataframe
def count_num(row):
    num = len( re.findall('[\d]', str(row)) )
    return num
df['numericals'] = df['url'].apply(lambda row: count_num(str(row)))

In [16]:
#all the unique special characters found in the urls in the dataframe
sp=[]
for row in df.itertuples():
    url=str(row.url)
    for i in url:
        if i.isalnum()==False:
            if i not in sp:
                sp.append(i)
print(sp)
            

['.', '/', '_', '-', '?', '=', '%', ':', '#', '(', ')', '+', ',', '&', '~', ';', '!', "'", '\\', ' ', '@', '$', '*', '|', '{', '}', '[', ']', '`', '^', '>']


In [17]:
#It was found that - . / ? = _ are usually found in the Phishing urls, 
#so new columns are created showing the count of each of the special character count in each url

#Count of the usually found special characters other than above mentioned characters 
#are generalized into a column called miscellaneous character

#This is done by using REGULAR EXPRESSION concept

def count_misc_ch(row):
    sc=0
    sc = sc+len( re.findall("'", str(row)) )
    sc = sc+len( re.findall("\\\\", str(row)) )
    sc = sc+len( re.findall("%", str(row)) )
    sc = sc+len( re.findall("&", str(row)) )
    sc = sc+len( re.findall(";", str(row)) )
    sc = sc+len( re.findall(":", str(row)) )
    sc = sc+len( re.findall("#", str(row)) )
    sc = sc+len( re.findall("~", str(row)) )
    sc = sc+len( re.findall(",", str(row)) )
    sc = sc+len( re.findall("}", str(row)) )
    sc = sc+len( re.findall("{", str(row)) )
    sc = sc+len( re.findall("\+", str(row)) )
    sc = sc+len( re.findall("\)", str(row)) )
    sc = sc+len( re.findall("\(", str(row)) )
    sc = sc+len( re.findall("!", str(row)) )
    sc = sc+len( re.findall(" ", str(row)) )
    sc = sc+len( re.findall(">", str(row)) )
    sc = sc+len( re.findall("\^", str(row)) )
    sc = sc+len( re.findall("\$", str(row)) )
    sc = sc+len( re.findall("\|", str(row)) )
    sc = sc+len( re.findall("\*", str(row)) )
    sc = sc+len( re.findall("\[", str(row)) )
    sc = sc+len( re.findall("//", str(row)) )
    sc = sc+len( re.findall("@", str(row)) )
    return sc
df['misc_ch'] = df['url'].apply(lambda row: count_misc_ch(row))

def count_d(row):
    sc = len( re.findall("-", str(row)) )
    return sc
df['dash'] = df['url'].apply(lambda row: count_d(row))

def count_dot(row):
    sc = len( re.findall("\.", str(row)) )
    return sc
df['dot'] = df['url'].apply(lambda row: count_dot(row))

def count_slash(row): 
    sc = len( re.findall("/", str(row)) )
    m = len( re.findall("//", str(row)) )
    sc=sc-(m*2)
    return sc
df['slash'] = df['url'].apply(lambda row: count_slash(row))

def count_qy(row):
    sc = len( re.findall("\?", str(row)) )
    return sc
df['quest'] = df['url'].apply(lambda row: count_qy(row))

def count_eq(row):
    sc = len( re.findall("=", str(row)) )
    return sc
df['equal'] = df['url'].apply(lambda row: count_eq(row))

def count_un(row):
    sc = len( re.findall("_", str(row)) )
    return sc
df['underscore'] = df['url'].apply(lambda row: count_un(row))


In [18]:
df.head()

,url,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore
0,www.crestonwood.com/router.php,0,0,0,0,3,1,0,0,0
1,rgipt.ac.in,0,0,0,0,2,0,0,0,0
2,vamoaestudiarmedicina.blogspot.com/,0,0,0,0,2,1,0,0,0
3,www.astrologyonline.eu/Astro_MemoNew/Profilo.asp,0,0,0,0,3,2,0,0,1
4,www.missfiga.com/,0,0,0,0,2,1,0,0,0


In [19]:
df.describe()

,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore
count,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000
mean,0.521691,8.572765,1.156649,1.315868,3.059715,2.987308,0.241187,0.467874,0.321674
std,0.499531,19.687067,4.669087,2.655505,2.320389,2.048341,0.523139,1.239488,1.025502
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000
75%,1.000000,8.000000,0.000000,1.000000,3.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1204.000000,427.000000,43.000000,37.000000,41.000000,17.000000,39.000000,79.000000


In [20]:
#Length of each url is calculated and recorded in a new column called count_len
#This feature is important bcz of usually found variations in the length of phishing urls and legitimate urls.
#Phishing urls tend to have more length

def count_len(row):
    l = len( row )
    return l
df['length_url'] = df['url'].apply(lambda row: count_len(str(row)))

In [21]:
df.head()

,url,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore,length_url
0,www.crestonwood.com/router.php,0,0,0,0,3,1,0,0,0,30
1,rgipt.ac.in,0,0,0,0,2,0,0,0,0,11
2,vamoaestudiarmedicina.blogspot.com/,0,0,0,0,2,1,0,0,0,35
3,www.astrologyonline.eu/Astro_MemoNew/Profilo.asp,0,0,0,0,3,2,0,0,1,48
4,www.missfiga.com/,0,0,0,0,2,1,0,0,0,17


In [22]:
#calculating Domain length and path length. Phishing urls tend to have more length
#Domain and path are extracted from the url using a pattern and REGULAR EXPRESSIONS
#A general pattern resembling a domain and path in url is defined using which they are extracted from url and length is calculated
#As it was not possoble to go through each and every url. If there are any unusual url, those are handled using try except block

def domain_len(row):
    pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
    try:
        m=re.match(pattern,row)
        l = len(m.group(2))
    except:
        l=0
    return l
df['domain_length'] = df['url'].apply(lambda row: domain_len(row))

def path_len(row):
    pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
    try:
        m=re.match(pattern,row)
        l = len(m.group(3))
    except:
        l=0
    return l
df['path_length'] = df['url'].apply(lambda row: path_len(row))

In [23]:
#Inorder to find how many of the urls in the dataframe have protocol or no... 
# A binary column is created to check the presence of the protocol in the url.
#Groups under RE concept is used to do the same
'''
def protocol(row):
    pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
    try:
        m=re.match(pattern,row)
        l = len(m.group(1))
        if(l>0):
            return 1
        else:
            return 0
    except:
        return 0
    return 0
df['protocol'] = df['url'].apply(lambda row: protocol(row))
'''

"\ndef protocol(row):\n    pattern=r'(ftps?://|https?://)?([^/]*\\.?[^/]+\\.[^/]+)(/*.*)'\n    try:\n        m=re.match(pattern,row)\n        l = len(m.group(1))\n        if(l>0):\n            return 1\n        else:\n            return 0\n    except:\n        return 0\n    return 0\ndf['protocol'] = df['url'].apply(lambda row: protocol(row))\n"

In [24]:
#df['protocol'].value_counts() # as we can see more than 85% of the data does not have the protocol

In [25]:
#Inorder to find the presence of brand names in the url. 

In [26]:
brand = pd.read_csv('top500Domains.csv')
brand.head()
"""We are importing another dataset which contains 500 top domains like google.com, yahoo.com etc,
we extract the domain names from each url in order to count brand names in each URL of our dataset. we store all the 
domain names in a set to avoid repition of any brand name. We also use this dataset of top domains to extract top 
level domain like .com,.net etc and store them in a set as well. In phishing URLs brand names are repeated more than once in 
subdomain or path of a URL."""


'We are importing another dataset which contains 500 top domains like google.com, yahoo.com etc,\nwe extract the domain names from each url in order to count brand names in each URL of our dataset. we store all the \ndomain names in a set to avoid repition of any brand name. We also use this dataset of top domains to extract top \nlevel domain like .com,.net etc and store them in a set as well. In phishing URLs brand names are repeated more than once in \nsubdomain or path of a URL.'

In [27]:
brand_df=[]
pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
for i in brand.itertuples():
    url = i.url
    try:
        m=re.match(pattern,url)
        brand_df.append(m.group(2))
    except:
        brand_df.append(0)

In [28]:
brand_tok = pd.DataFrame(brand_df,columns =['Domain'])
brand_tok.head(15)

,Domain
0,www.google.com
1,youtube.com
2,apple.com
3,microsoft.com
4,www.blogger.com
5,support.google.com
6,cloudflare.com
7,play.google.com
8,docs.google.com
9,youtu.be


In [29]:
import re
import nltk
import string

brand_tok['tok'] = brand_tok['Domain'].str.replace("[^a-zA-Z#]", " ")
brand_tok['tok'] = brand_tok['tok'].str.replace("\s+", " ")
brand_tok.head()

,Domain,tok
0,www.google.com,www google com
1,youtube.com,youtube com
2,apple.com,apple com
3,microsoft.com,microsoft com
4,www.blogger.com,www blogger com


In [30]:
"""In order to fetch the domain names from the entire url of top500domains, we first converted our URLs in tokens
then removed the third level domain and top level domain with the help of tldextract"""
from tldextract import extract
import tldextract
third_ld=[]
for url in brand_df:
    results = extract(url)
    subdomain = results.subdomain
    if subdomain:
        # Just get the 3rd level domain:
        third_level_domain = subdomain.split('.')[-1]
        third_ld.append(third_level_domain);
        #print(third_level_domain)
        

In [31]:
third_ld = set(third_ld)

Third level domains 

In [32]:
brand_names=[]
for url in brand_df:
    extracted = tldextract.extract(url)
    brand_names.append("{}".format(extracted.domain))
brand_names_additional = ['flipkart','google','yahoo','ebay','passportindia','ieee','amazon','godaddy','alibaba','zappos','aliexpress','rakuten','walmart','target','geeksforgeeks','leetcode','makemytrip','bookmyshow','zomato','swiggy','impartus','axisbank','icicibank','sciencedirect']
brand_names = brand_names + brand_names_additional

In [33]:
top_ld=[]
for url in brand_df:
    extracted = tldextract.extract(url)
    top_ld.append("{}".format(extracted.suffix))

In [34]:
top_ld=set(top_ld)

In [35]:
brand_names = set(brand_names)

In [36]:
brand_names = brand_names-third_ld-top_ld

In [37]:
len(brand_names)

414

In [38]:
brand_names

{'20minutos',
 '4shared',
 'a8',
 'abc',
 'about',
 'aboutads',
 'abril',
 'academia',
 'addthis',
 'addtoany',
 'adobe',
 'afternic',
 'akamaihd',
 'alexa',
 'alibaba',
 'aliexpress',
 'allaboutcookies',
 'allrecipes',
 'amazon',
 'ameblo',
 'android',
 'aol',
 'apache',
 'apple',
 'archive',
 'archives',
 'arstechnica',
 'asus',
 'axisbank',
 'bandcamp',
 'bbc',
 'behance',
 'berkeley',
 'biglobe',
 'bing',
 'bit',
 'bitly',
 'blackberry',
 'blogger',
 'bloglovin',
 'blogspot',
 'bloomberg',
 'booking',
 'bookmyshow',
 'box',
 'brandbucket',
 'britannica',
 'bund',
 'businessinsider',
 'buydomains',
 'buzzfeed',
 'cam',
 'cambridge',
 'canalblog',
 'cbc',
 'cbslocal',
 'cbsnews',
 'cdc',
 'change',
 'channel4',
 'chicagotribune',
 'cia',
 'cisco',
 'clickbank',
 'cloudflare',
 'cmu',
 'cnbc',
 'cnet',
 'cnn',
 'columbia',
 'cornell',
 'coursera',
 'cpanel',
 'creativecommons',
 'dailymail',
 'dailymotion',
 'dan',
 'daum',
 'debian',
 'deezer',
 'dell',
 'depositfiles',
 'detik',
 'd

Suspicious Words

In [39]:
suspicious_words = ['g00gle','yah00','googles','amp','php','index','id','openid','wp-content','aspx','email','battle','cmd','webscr','images','d3',"confirm", "account", "secure", "webscr", "login", "signin", "submit", "update", "logon", "secure", "wp", "cmd","admin","approval", "authority", "balance", "amount"]

"""we created a list of suspicious words like confirm, webscr, login etc. and later compared it with all URLs in our dataset
and counted the number of suspicious words in each url because such words are commonly used in phishing urls 
"""

'we created a list of suspicious words like confirm, webscr, login etc. and later compared it with all URLs in our dataset\nand counted the number of suspicious words in each url because such words are commonly used in phishing urls \n'

In [40]:
suspicious_words = set(suspicious_words)

In [41]:
len(suspicious_words)

30

Feature extraction for brand names,suspicious words and TLD 

In [42]:
l=[]
pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
for i in df.itertuples():
    url = i.url
    try:
        m=re.match(pattern,url)
        l.append(m.group(2))
    except:
        l.append(0)

In [43]:
df['tokens'] = df['url'].str.replace("[^a-zA-Z#]", " ")
df['tokens'] = df['tokens'].str.replace("\s+", " ")
df['tokens']=df['tokens'].fillna("")
df.head()

,url,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore,length_url,domain_length,path_length,tokens
0,www.crestonwood.com/router.php,0,0,0,0,3,1,0,0,0,30,19,11,www crestonwood com router php
1,rgipt.ac.in,0,0,0,0,2,0,0,0,0,11,11,0,rgipt ac in
2,vamoaestudiarmedicina.blogspot.com/,0,0,0,0,2,1,0,0,0,35,34,1,vamoaestudiarmedicina blogspot com
3,www.astrologyonline.eu/Astro_MemoNew/Profilo.asp,0,0,0,0,3,2,0,0,1,48,22,26,www astrologyonline eu Astro MemoNew Profilo asp
4,www.missfiga.com/,0,0,0,0,2,1,0,0,0,17,16,1,www missfiga com


In [44]:
import re
def match(input_string, string_list):
    words = re.findall(r'\w+', input_string)
    return [word for word in words if word in string_list]


"""the function brandCount, checks for match of tokens of the URL with the brand name and if they match the brandCount 
for that URL is incremented by one."""
def brandCount(tokens):
    l = match(tokens, brand_names)
    return len(l)
df['brandCount'] = df['tokens'].apply(lambda row: brandCount(row))

In [45]:
df.head(10)

,url,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore,length_url,domain_length,path_length,tokens,brandCount
0,www.crestonwood.com/router.php,0,0,0,0,3,1,0,0,0,30,19,11,www crestonwood com router php,1
1,rgipt.ac.in,0,0,0,0,2,0,0,0,0,11,11,0,rgipt ac in,0
2,vamoaestudiarmedicina.blogspot.com/,0,0,0,0,2,1,0,0,0,35,34,1,vamoaestudiarmedicina blogspot com,1
3,www.astrologyonline.eu/Astro_MemoNew/Profilo.asp,0,0,0,0,3,2,0,0,1,48,22,26,www astrologyonline eu Astro MemoNew Profilo asp,0
4,www.missfiga.com/,0,0,0,0,2,1,0,0,0,17,16,1,www missfiga com,0
5,www.2345daohang.com/,0,4,0,0,2,1,0,0,0,20,19,1,www daohang com,0
6,www.game.co.uk/en/games/nintendo-switch/ninten...,0,0,0,2,3,5,0,0,0,56,14,42,www game co uk en games nintendo switch ninten...,0
7,blog.hubspot.com/marketing/email-open-click-ra...,0,0,0,4,2,2,0,0,0,58,16,42,blog hubspot com marketing email open click ra...,0
8,sophie-world.com/games/port-and-starboard,0,0,0,3,1,2,0,0,0,41,16,25,sophie world com games port and starboard,0
9,www.provenancevineyards.com/,0,0,0,0,2,1,0,0,0,28,27,1,www provenancevineyards com,0


In [46]:
"""the function suspiciousCount, checks for match of tokens of the URL with the suspicious words and if they match the
SuspiciousCount for that URL is incremented by one."""
def suspiciousCount(tokens):
    l = match(tokens, suspicious_words)
    return len(l)
df['suspiciousCount'] = df['tokens'].apply(lambda row: suspiciousCount(row))

Count of common Top level domains

In [47]:
common_top_ld = ['com','ru','org','net','info','biz','online','blog','travel','web','app','edu','gov','uk','ca','jp','fr','au','us','de','ch','it','nl','se','no','es','mil']

In [48]:
common_top_ld=set(common_top_ld)

In [49]:
"""the function common_top_ld_Count, checks for match of tokens of the URL with the common top level domains and if they match 
the common_top_ld_Count for that URL is incremented by one."""
def common_top_ld_Count(tokens):
    l = match(tokens, common_top_ld)
    return len(l)
df['comm_tldCount'] = df['tokens'].apply(lambda row: common_top_ld_Count(row))

# Domain from URL (extraction)

In [50]:
#getting the tokens of domain
import re
import nltk
import string

pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
def dom(url):
    try:
        m=re.match(pattern,url)
        return (m.group(2))#inorder to get the domain of the urls the 2nd subgroup of the pattern has to be taken
    except:
        return 0
df['domain'] = df['url'].apply(lambda row: dom(row)) #and assigning that extracted feature to the dataframe 

df['tok'] = df['domain'].str.replace("[^a-zA-Z#]", " ")
df['tok'] = df['tok'].str.replace("\s+", " ")
df['tok']=df['tok'].fillna("")

In [51]:
def no_tok(tok): #counts the number of tokens in the domain
    if tok=='':
        return 0
    else:
        l=[]
        c=0
        l = tok.split()
        c = len(l)
        return c

def lar_tok(tok): # finds the lenght of the largest token present in the domain of a url 
    l=[]
    ml = 0
    tl = 0
    text = tok
    l = text.split()
    c = len(l)
    for t in l:
        tl = tl + len(t)
        if (ml<len(t)):
            ml = len(t)
    return ml

def avg_dom_tok_len(tok): # finds the average token length in a url 
    ml = 0
    tl = 0
    text = tok
    l = text.split()
    c = len(l)
    for t in l:
        tl = tl + len(t)
        if (ml<len(t)):
            ml = len(t)
    if c==0:
        return 0
    else:
        return (tl/c)
   
#assiging the columns to above extracted features
df['no_tok_dom'] = df['tok'].apply(lambda row: no_tok(row))
df['dom_large_tok_len'] = df['tok'].apply(lambda row: lar_tok(row))
df['avg_dom_tok_len'] = df['tok'].apply(lambda row: avg_dom_tok_len(row))
df['is_IP'] = df['no_tok_dom'].apply(lambda row: 1 if row==0 else 0)

In [52]:
df.head()

,url,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore,...,tokens,brandCount,suspiciousCount,comm_tldCount,domain,tok,no_tok_dom,dom_large_tok_len,avg_dom_tok_len,is_IP
0,www.crestonwood.com/router.php,0,0,0,0,3,1,0,0,0,...,www crestonwood com router php,1,1,1,www.crestonwood.com,www crestonwood com,3,11,5.666667,0
1,rgipt.ac.in,0,0,0,0,2,0,0,0,0,...,rgipt ac in,0,0,0,rgipt.ac.in,rgipt ac in,3,5,3.000000,0
2,vamoaestudiarmedicina.blogspot.com/,0,0,0,0,2,1,0,0,0,...,vamoaestudiarmedicina blogspot com,1,0,1,vamoaestudiarmedicina.blogspot.com,vamoaestudiarmedicina blogspot com,3,21,10.666667,0
3,www.astrologyonline.eu/Astro_MemoNew/Profilo.asp,0,0,0,0,3,2,0,0,1,...,www astrologyonline eu Astro MemoNew Profilo asp,0,0,0,www.astrologyonline.eu,www astrologyonline eu,3,15,6.666667,0
4,www.missfiga.com/,0,0,0,0,2,1,0,0,0,...,www missfiga com,0,0,1,www.missfiga.com,www missfiga com,3,8,4.666667,0


In [53]:
pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'
def path(url):
    try:
        m=re.match(pattern,url)
        return (m.group(3)) #takes the 3rd subgroup from the above pattern which is the path incase of urls
    except:
        return 0
df['path'] = df['url'].apply(lambda row: path(row)) # adding paths to the defined dataframe

df['path_tok'] = df['path'].str.replace("[^a-zA-Z#]", " ")
df['path_tok'] = df['path_tok'].str.replace("\s+", " ")
df['path_tok']=df['path_tok'].fillna("")

def avg_path_tok_len(tok): #finding the average lenth of path 
    ml = 0
    tl = 0
    text = tok
    l = text.split()
    c = len(l)
    for t in l:
        tl = tl + len(t)
        if (ml<len(t)):
            ml = len(t)
    if c==0:
        return 0
    else:
        return (tl/c)
    
df['avg_path_tok_len'] = df['path_tok'].apply(lambda row: avg_dom_tok_len(row)) #assigning the average length data(array) to the dataframe


In [54]:
df = df.drop(['domain','tok','path','path_tok'],axis='columns')#dropping the columns which arent considered

Directories in URL

In [55]:
pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)' #finding the count of directories found in the url .
        
def dire_count(url):
    try:
        m=re.match(pattern,url)
        text=m.group(3) # the path subgroup is taken since it contains the directories and the file. 
        cou=len((re.findall(r'[^/]+/', text))) #regular expression to find the number of directories in a url
        return cou
    except:
        #print(i)
        return 0
df['dir_count'] = df['url'].apply(lambda row: dire_count(row))# assigning the data in the dataframe

In [56]:
#finding the number of digits present in domain
pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)'         
def domain_digit_count(url):
    try:
        dom=re.match(pattern,url)
        dom=dom.group(2)
        dom_count=len(re.findall('\d', dom)) #regular expression to find the number of digits 
        return dom_count
    except:
        return 0
df['domain_digit_count'] = df['url'].apply(lambda row: domain_digit_count(row))

In [57]:
#to check the presence of http/https in domain
pattern=r'(ftps?://|https?://)?([^/]*\.?[^/]+\.[^/]+)(/*.*)' 
def presence_http(url):  
    try:
        http=re.match(pattern,url)
        http=http.group(2)
        if(re.findall("http|https", http)) :#regular expression to find http/https
            return 1
        else:
            return 0    
    except:
        return 0
df['presence_http'] = df['url'].apply(lambda row: presence_http(row))

In [58]:
df.head()

,url,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore,...,suspiciousCount,comm_tldCount,no_tok_dom,dom_large_tok_len,avg_dom_tok_len,is_IP,avg_path_tok_len,dir_count,domain_digit_count,presence_http
0,www.crestonwood.com/router.php,0,0,0,0,3,1,0,0,0,...,1,1,3,11,5.666667,0,4.5,0,0,0
1,rgipt.ac.in,0,0,0,0,2,0,0,0,0,...,0,0,3,5,3.000000,0,0.0,0,0,0
2,vamoaestudiarmedicina.blogspot.com/,0,0,0,0,2,1,0,0,0,...,0,1,3,21,10.666667,0,0.0,0,0,0
3,www.astrologyonline.eu/Astro_MemoNew/Profilo.asp,0,0,0,0,3,2,0,0,1,...,0,0,3,15,6.666667,0,5.5,1,0,0
4,www.missfiga.com/,0,0,0,0,2,1,0,0,0,...,0,1,3,8,4.666667,0,0.0,0,0,0


In [59]:
df.describe()

,label,numericals,misc_ch,dash,dot,slash,quest,equal,underscore,length_url,...,suspiciousCount,comm_tldCount,no_tok_dom,dom_large_tok_len,avg_dom_tok_len,is_IP,avg_path_tok_len,dir_count,domain_digit_count,presence_http
count,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,...,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000,131574.000000
mean,0.521691,8.572765,1.156649,1.315868,3.059715,2.987308,0.241187,0.467874,0.321674,68.872330,...,0.980786,1.326485,3.723266,9.055490,5.148177,0.002713,4.568345,2.013764,1.380949,0.001178
std,0.499531,19.687067,4.669087,2.655505,2.320389,2.048341,0.523139,1.239488,1.025502,64.119484,...,2.042277,1.116231,4.099443,4.407958,1.931290,0.052019,2.554173,2.029264,7.046402,0.034303
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,32.000000,...,0.000000,1.000000,2.000000,6.000000,3.800000,0.000000,3.250000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,48.000000,...,0.000000,1.000000,3.000000,8.000000,4.750000,0.000000,4.500000,1.000000,0.000000,0.000000
75%,1.000000,8.000000,0.000000,1.000000,3.000000,4.000000,0.000000,0.000000,0.000000,83.000000,...,1.000000,1.000000,4.000000,11.000000,6.000000,0.000000,5.818182,3.000000,0.000000,0.000000
max,1.000000,1204.000000,427.000000,43.000000,37.000000,41.000000,17.000000,39.000000,79.000000,2669.000000,...,107.000000,27.000000,78.000000,62.000000,24.666667,1.000000,160.000000,43.000000,134.000000,1.000000


In [60]:
df.columns

Index(['url', 'label', 'numericals', 'misc_ch', 'dash', 'dot', 'slash',
       'quest', 'equal', 'underscore', 'length_url', 'domain_length',
       'path_length', 'tokens', 'brandCount', 'suspiciousCount',
       'comm_tldCount', 'no_tok_dom', 'dom_large_tok_len', 'avg_dom_tok_len',
       'is_IP', 'avg_path_tok_len', 'dir_count', 'domain_digit_count',
       'presence_http'],
      dtype='object')

# Split

In [61]:
df1=df

In [62]:
#df1['label'].replace(to_replace=['1', '0'], value=[1, 0], inplace=True)

In [63]:
df1 = df1.drop(['tokens'],axis='columns')

In [64]:
y = df1['label']
x = df1.drop(['label'],axis='columns')

In [65]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [66]:
x_train_url = x_train['url']
x_train = x_train.drop(['url'],axis='columns')

x_test_url = x_test['url']
x_test = x_test.drop(['url'],axis='columns')

In [67]:
x_train_url

21449                   news.cnet.com/2100-1017-237759.html
128898    www.smxszz.com/images/?us.battle.net/login/en/...
37371     www-306.ibm.com/software/tivoli/solutions/secu...
113685    docs.google.com/spreadsheet/viewform?formkey=d...
4035                                  www.petroczigabor.hu/
                                ...                        
129984    wellfargo.kvt8vf7klcgp7abb5yhgcw.forcadomar.co...
91128     mariachisadomicilio.cl/images/ben/01ccb5def9db...
40865                                 launchpad.net/rubber/
23901     www.theregister.co.uk/2000/10/04/napster_will_...
120849    '9d345009-a-62cb3a1a-s-sites.googlegroups.com/...
Name: url, Length: 92101, dtype: object

# Data transformation

In [68]:
x.columns

Index(['url', 'numericals', 'misc_ch', 'dash', 'dot', 'slash', 'quest',
       'equal', 'underscore', 'length_url', 'domain_length', 'path_length',
       'brandCount', 'suspiciousCount', 'comm_tldCount', 'no_tok_dom',
       'dom_large_tok_len', 'avg_dom_tok_len', 'is_IP', 'avg_path_tok_len',
       'dir_count', 'domain_digit_count', 'presence_http'],
      dtype='object')

In [69]:
x['brandCount'].describe()

count    131574.000000
mean          0.490294
std           0.771693
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max          19.000000
Name: brandCount, dtype: float64

In [70]:
#df2 = (df2-df2.mean())/df2.std()

mean_train = x_train.mean()
std_dev_train = x_train.std()

x_train = (x_train - mean_train) / std_dev_train

In [71]:
x_train['brandCount'].describe()

count    9.210100e+04
mean    -1.636576e-15
std      1.000000e+00
min     -6.327331e-01
25%     -6.327331e-01
50%     -6.327331e-01
75%      6.536701e-01
max      2.380893e+01
Name: brandCount, dtype: float64

In [72]:
x_test = (x_test - mean_train) / std_dev_train

In [73]:
mean_train

numericals             8.631090
misc_ch                1.158663
dash                   1.316490
dot                    3.057024
slash                  2.987872
quest                  0.242364
equal                  0.468529
underscore             0.320594
length_url            68.870240
domain_length         21.370680
path_length           47.499560
brandCount             0.491862
suspiciousCount        0.984018
comm_tldCount          1.327749
no_tok_dom             3.720513
dom_large_tok_len      9.043431
avg_dom_tok_len        5.146645
is_IP                  0.002617
avg_path_tok_len       4.570461
dir_count              2.014191
domain_digit_count     1.387227
presence_http          0.001249
dtype: float64

In [74]:
std_dev_train

numericals            19.966955
misc_ch                4.564997
dash                   2.651285
dot                    2.317864
slash                  2.042362
quest                  0.525526
equal                  1.242227
underscore             1.033861
length_url            63.949401
domain_length         21.188167
path_length           58.292116
brandCount             0.777361
suspiciousCount        2.088718
comm_tldCount          1.119973
no_tok_dom             4.116755
dom_large_tok_len      4.402245
avg_dom_tok_len        1.929077
is_IP                  0.051087
avg_path_tok_len       2.571250
dir_count              2.022599
domain_digit_count     7.096429
presence_http          0.035314
dtype: float64

# URL ENCODING

In [75]:
char = sorted(char)
vocab = dict((c, i) for i, c in enumerate(char)) #dictionary of letters which has ascii values
sequences = list()
for i in vocab:
     if i in vocab: 
            vocab[i] +=1
            

In [76]:
#vocab have all unique characters that have ascii values, but to tackle an unknown character, we use tokenizer
from keras.preprocessing.text import Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK') 
#oov_token='UNK' is a inbuilt option to deal with unknown characters in the text
tk.word_index = vocab.copy()  
#instead of have 2 vocab we copy the vocab created into tok so has to introduce unkown var
tk.word_index[tk.oov_token] = max(vocab.values()) + 1
#tk.word_index

In [77]:
vocab_size = len(tk.word_index)+1 #0 is left for padding
max_length = 1000 #max length from our dataset is 2k but we have used max length 1000 so any characters after that will be discarded

In [78]:
tk.word_index

{'\t': 1,
 '\n': 2,
 '\x0b': 3,
 '\x0c': 4,
 '\r': 5,
 ' ': 6,
 '!': 7,
 '"': 8,
 '#': 9,
 '$': 10,
 '%': 11,
 '&': 12,
 "'": 13,
 '(': 14,
 ')': 15,
 '*': 16,
 '+': 17,
 ',': 18,
 '-': 19,
 '.': 20,
 '/': 21,
 '0': 22,
 '1': 23,
 '2': 24,
 '3': 25,
 '4': 26,
 '5': 27,
 '6': 28,
 '7': 29,
 '8': 30,
 '9': 31,
 ':': 32,
 ';': 33,
 '<': 34,
 '=': 35,
 '>': 36,
 '?': 37,
 '@': 38,
 'A': 39,
 'B': 40,
 'C': 41,
 'D': 42,
 'E': 43,
 'F': 44,
 'G': 45,
 'H': 46,
 'I': 47,
 'J': 48,
 'K': 49,
 'L': 50,
 'M': 51,
 'N': 52,
 'O': 53,
 'P': 54,
 'Q': 55,
 'R': 56,
 'S': 57,
 'T': 58,
 'U': 59,
 'V': 60,
 'W': 61,
 'X': 62,
 'Y': 63,
 'Z': 64,
 '[': 65,
 '\\': 66,
 ']': 67,
 '^': 68,
 '_': 69,
 '`': 70,
 'a': 71,
 'b': 72,
 'c': 73,
 'd': 74,
 'e': 75,
 'f': 76,
 'g': 77,
 'h': 78,
 'i': 79,
 'j': 80,
 'k': 81,
 'l': 82,
 'm': 83,
 'n': 84,
 'o': 85,
 'p': 86,
 'q': 87,
 'r': 88,
 's': 89,
 't': 90,
 'u': 91,
 'v': 92,
 'w': 93,
 'x': 94,
 'y': 95,
 'z': 96,
 '{': 97,
 '|': 98,
 '}': 99,
 '~': 100

In [79]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = tk.texts_to_sequences(x_train_url) #converting text into integer encoding
x_train_url = pad_sequences(sequences, maxlen=max_length) #pad text to have same length as max length

sequences = tk.texts_to_sequences(x_test_url) #converting text into integer encoding
x_test_url = pad_sequences(sequences, maxlen=max_length) #pad text to have same length as max length

In [80]:
x_train_url[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [81]:
#creating embedding weights which will be used in embedding layer in the model
#this will convert the input interger encoded text into one hot encoding
embedded_weights = []
embedded_weights.append(np.zeros(vocab_size))

for char,i in tk.word_index.items():
    one_hot = np.zeros(vocab_size)
    one_hot[i-1]=1
    embedded_weights.append(one_hot)
    
embedded_weights = np.array(embedded_weights)
print(embedded_weights.shape)
print(embedded_weights)

(102, 102)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


# Model

In [82]:

from keras.layers import Dense, Dropout, Activation,Input, MaxPooling1D, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras import callbacks
from tensorflow.python.keras import models, layers, optimizers,Model

#from keras.optimizers import RMSprop 
from keras.layers.convolutional import Conv1D
from keras.layers import concatenate

In [83]:
conv_lay = [[256,5,3],[256,7,-1]] #,[256,7,-1],[256,6,3],[256,7,-1],[256,7,3]

In [84]:
ip_feat = Input((22,))
ip1 = Dense(60)(ip_feat)
ip1 = Activation('relu')(ip1)

url_enco = Input(shape=(1000,),dtype='int64')
x = Embedding(vocab_size,102,input_length=max_length,weights = [embedded_weights])(url_enco)

for fil_num,fil_size,pool_size in conv_lay:
    x = Conv1D(fil_num,fil_size)(x)
    x = Activation('relu')(x)
    if pool_size != -1:
        x = MaxPooling1D(pool_size=pool_size)(x)
x = Flatten()(x)
ip2 = Dense(60)(x)
ip2 = Activation('relu')(ip2)

L = concatenate([ip1,ip2])

L1 = Dense(60)(L)
L1 = Activation('swish')(L1)
L2 = Dense(60)(L1)
L2 = Activation('swish')(L2)
L3 = Dense(120)(L2)
L3 = Activation('swish')(L3)
L4 = Dense(120)(L3)
L4 = Activation('swish')(L4)
L5 = Dense(120)(L4)
L5 = Activation('swish')(L5)
L6 = Dense(60)(L5)
L6 = Activation('swish')(L6)
L6 = Dropout(0.3,seed=2)(L6)
L7 = Dense(1)(L6)
op = Activation('sigmoid')(L7)
model = Model([ip_feat,url_enco],[op])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1000, 102)    10404       input_2[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 996, 256)     130816      embedding[0][0]                  
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 996, 256)     0           conv1d[0][0]                     
______________________________________________________________________________________________

In [85]:
#opt = RMSprop(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

earlystopping = callbacks.EarlyStopping(monitor ="val_loss", mode='min', patience = 5, 
                                        restore_best_weights = True)

In [86]:
history = model.fit([x_train,x_train_url], y_train, epochs=50, batch_size=64, validation_data=([x_test,x_test_url], y_test), verbose=2,callbacks =[earlystopping])

Epoch 1/50
1439/1439 - 2155s - loss: 0.2008 - accuracy: 0.9200 - val_loss: 0.1115 - val_accuracy: 0.9577
Epoch 2/50
1439/1439 - 2144s - loss: 0.1032 - accuracy: 0.9639 - val_loss: 0.1081 - val_accuracy: 0.9629
Epoch 3/50
1439/1439 - 2125s - loss: 0.0778 - accuracy: 0.9735 - val_loss: 0.1963 - val_accuracy: 0.9524
Epoch 4/50
1439/1439 - 2109s - loss: 0.0657 - accuracy: 0.9788 - val_loss: 0.0974 - val_accuracy: 0.9669
Epoch 5/50
1439/1439 - 2118s - loss: 0.0592 - accuracy: 0.9818 - val_loss: 0.1617 - val_accuracy: 0.9667
Epoch 6/50
1439/1439 - 2088s - loss: 0.0499 - accuracy: 0.9841 - val_loss: 0.1596 - val_accuracy: 0.9663
Epoch 7/50
1439/1439 - 2077s - loss: 0.0510 - accuracy: 0.9854 - val_loss: 0.3808 - val_accuracy: 0.9103
Epoch 8/50
1439/1439 - 2067s - loss: 0.0519 - accuracy: 0.9866 - val_loss: 0.2338 - val_accuracy: 0.9593
Epoch 9/50
1439/1439 - 2070s - loss: 0.0440 - accuracy: 0.9877 - val_loss: 0.1298 - val_accuracy: 0.9665


In [87]:
loss, acc = model.evaluate([x_test,x_test_url], y_test, verbose=2)

1234/1234 - 209s - loss: 0.0974 - accuracy: 0.9669


In [88]:
loss, acc = model.evaluate([x_train,x_train_url], y_train, verbose=2)

2878/2878 - 491s - loss: 0.0361 - accuracy: 0.9888


In [92]:
from keras.models import load_model
#model.save("Review3_DNNCNN.h5")

In [91]:
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from mlxtend.evaluate import bias_variance_decomp
from sklearn.metrics import zero_one_loss
from sklearn.metrics import confusion_matrix

#y_test = y_test.astype('int64')
y_test = y_test.astype('float64')

acnn_predict = model.predict([x_test,x_test_url])
acnn_predict=(acnn_predict>0.5)
#print(type(cnn_predict))
print(confusion_matrix(y_test, acnn_predict))
print(classification_report(y_test,acnn_predict))

accuracy = accuracy_score(y_test,acnn_predict)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test,acnn_predict)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test,acnn_predict)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test,acnn_predict)
print('F1 score: %f' % f1)

auc = roc_auc_score(y_test,acnn_predict)
print('ROC AUC: %f' % auc)

print('Mean_abso_mean : ', mean_absolute_error(y_test,acnn_predict))
print('Mean_sq_mean : ', mean_squared_error(y_test,acnn_predict))

[[18382   489]
 [  818 19771]]
              precision    recall  f1-score   support

         0.0       0.96      0.97      0.97     18871
         1.0       0.98      0.96      0.97     20589

    accuracy                           0.97     39460
   macro avg       0.97      0.97      0.97     39460
weighted avg       0.97      0.97      0.97     39460

Accuracy: 0.966878
Precision: 0.975864
Recall: 0.960270
F1 score: 0.968004
ROC AUC: 0.967179
Mean_abso_mean :  0.03312214901165737
Mean_sq_mean :  0.03312214901165737
